In [6]:
import numpy as np
import pandas as pd
import panel as pn
import hvplot.pandas
import holoviews as hv
from bokeh.palettes import Category10
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')


In [ ]:

# Initialize Panel
pn.extension('tabulator', sizing_mode="stretch_width")
hv.extension('bokeh')

class FinancialDashboard:
    def __init__(self):
        # Generate sample data
        self.generate_data()
        # Initialize the dashboard components
        self.create_widgets()
        self.create_dashboard()
    
    def generate_data(self):
        """Generate sample financial data"""
        np.random.seed(42)
        dates = pd.date_range(start='2023-01-01', periods=252, freq='B')
        
        # Generate price series
        returns = np.random.normal(0.0005, 0.01, len(dates))
        price = 100 * np.exp(np.cumsum(returns))
        volume = np.random.lognormal(10, 1, len(dates))
        
        self.df = pd.DataFrame({
            'Date': dates,
            'Price': price,
            'Returns': returns,
            'Volume': volume,
            'Volatility': pd.Series(returns).rolling(20).std() * np.sqrt(252)
        })
        self.df.set_index('Date', inplace=True)
    
    def create_widgets(self):
        """Create interactive widgets"""
        # Time period selector
        self.period_selector = pn.widgets.Select(
            name='Time Period',
            options=['1M', '3M', '6M', 'YTD', '1Y', 'ALL'],
            value='6M'
        )
        
        # Technical indicator selector
        self.indicator_selector = pn.widgets.MultiChoice(
            name='Technical Indicators',
            options=['SMA', 'EMA', 'Bollinger Bands', 'RSI'],
            value=['SMA']
        )
        
        # Chart type selector
        self.chart_type = pn.widgets.Select(
            name='Chart Type',
            options=['Line', 'Candlestick', 'OHLC'],
            value='Line'
        )
    
    def calculate_metrics(self):
        """Calculate key financial metrics"""
        returns = self.df['Returns']
        
        metrics = {
            'Total Return': f"{(self.df['Price'][-1] / self.df['Price'][0] - 1) * 100:.2f}%",
            'Annual Volatility': f"{returns.std() * np.sqrt(252) * 100:.2f}%",
            'Sharpe Ratio': f"{np.mean(returns) / np.std(returns) * np.sqrt(252):.2f}",
            'Max Drawdown': f"{(self.df['Price'] / self.df['Price'].expanding(min_periods=1).max() - 1).min() * 100:.2f}%"
        }
        return metrics
    
    def create_price_plot(self):
        """Create main price plot with technical indicators"""
        df_plot = self.df.copy()
        
        # Add technical indicators
        if 'SMA' in self.indicator_selector.value:
            df_plot['SMA_20'] = df_plot['Price'].rolling(window=20).mean()
            df_plot['SMA_50'] = df_plot['Price'].rolling(window=50).mean()
        
        if 'EMA' in self.indicator_selector.value:
            df_plot['EMA_20'] = df_plot['Price'].ewm(span=20).mean()
        
        if 'Bollinger Bands' in self.indicator_selector.value:
            df_plot['BB_middle'] = df_plot['Price'].rolling(window=20).mean()
            df_plot['BB_upper'] = df_plot['BB_middle'] + 2 * df_plot['Price'].rolling(window=20).std()
            df_plot['BB_lower'] = df_plot['BB_middle'] - 2 * df_plot['Price'].rolling(window=20).std()
        
        # Create base price plot
        price_plot = df_plot['Price'].hvplot(
            title='Price Chart',
            height=400,
            line_width=2,
            color='blue'
        )
        
        # Add technical indicators to plot
        if 'SMA' in self.indicator_selector.value:
            price_plot *= df_plot['SMA_20'].hvplot(color='red', line_width=1)
            price_plot *= df_plot['SMA_50'].hvplot(color='green', line_width=1)
        
        if 'EMA' in self.indicator_selector.value:
            price_plot *= df_plot['EMA_20'].hvplot(color='purple', line_width=1)
        
        if 'Bollinger Bands' in self.indicator_selector.value:
            price_plot *= df_plot['BB_upper'].hvplot(color='gray', line_width=1)
            price_plot *= df_plot['BB_middle'].hvplot(color='gray', line_width=1)
            price_plot *= df_plot['BB_lower'].hvplot(color='gray', line_width=1)
        
        return price_plot
    
    def create_volume_plot(self):
        """Create volume subplot"""
        return self.df['Volume'].hvplot.bar(
            title='Volume',
            height=200,
            color='blue',
            alpha=0.5
        )
    
    def create_returns_plot(self):
        """Create returns distribution plot"""
        return self.df['Returns'].hvplot.hist(
            title='Returns Distribution',
            height=200,
            bins=50,
            color='blue',
            alpha=0.5
        )
    
    def create_volatility_plot(self):
        """Create volatility plot"""
        return self.df['Volatility'].hvplot(
            title='Rolling Volatility (20-day)',
            height=200,
            line_width=2,
            color='red'
        )
    
    def create_metrics_table(self):
        """Create metrics summary table"""
        metrics = self.calculate_metrics()
        return pn.pane.DataFrame(
            pd.DataFrame(metrics.items(), columns=['Metric', 'Value']),
            width=300,
            height=150
        )
    
    def create_dashboard(self):
        """Assemble all components into a dashboard"""
        # Header
        header = pn.Row(
            pn.pane.Markdown('# Financial Dashboard'),
            pn.Spacer(width=20),
            self.period_selector,
            self.indicator_selector,
            self.chart_type
        )
        
        # Main plots
        plots = pn.Column(
            self.create_price_plot,
            pn.Row(
                self.create_volume_plot,
                self.create_returns_plot
            ),
            self.create_volatility_plot
        )
        
        # Sidebar with metrics
        sidebar = pn.Column(
            pn.pane.Markdown('## Key Metrics'),
            self.create_metrics_table,
            width=300
        )
        
        # Combine all elements
        self.dashboard = pn.Column(
            header,
            pn.Row(plots, sidebar)
        )
    
    def get_dashboard(self):
        """Return the dashboard for display"""
        return self.dashboard

# Create and show dashboard
dashboard = FinancialDashboard()
op = dashboard.get_dashboard().servable()

# For Jupyter Notebook display


In [5]:
op.show()

Launching server at http://localhost:58201
